# Puntos que deben conectarse en Google API

In [1]:
import pandas as pd
import numpy as np
import requests
import json
import pickle

In [2]:
data_point = pd.read_csv('centros_googleApi')

In [3]:
data_point.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,26,27,28,29,30,31,32,33,34,cluster_point
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,"(-78.50040125, -0.20248826666666667)"
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"(-78.49079993333334, -0.21495733333333333)"
2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,"(-78.49941296, -0.20644891999999998)"
3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"(-78.4981667125, -0.21103442500000003)"
4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,"(-78.50411983333333, -0.20301313333333335)"


Método para obtener los puntos del dataset.

In [4]:
puntos = list(data_point['cluster_point'])

In [5]:
puntos[0].replace(')','').replace('(','').split(',')

['-78.50040125', ' -0.20248826666666667']

Creación de un diccionario donde cada punto se conecta con las coordenadas de sus vecinos.

In [6]:
longitud,latitud = puntos[0].replace(')','').replace('(','').split(',')

In [7]:
latitud

' -0.20248826666666667'

In [8]:
longitud

'-78.50040125'

In [9]:
diccionar_coordenadas = {}
listaDEpuntos = []
for punto in puntos:
    longitud,latitud = punto.replace(')','').replace('(','').split(',')    
    #google maps pide latitud primero
    coordenada = (float(latitud),float(longitud))
    listaDEpuntos.append(coordenada)
    diccionar_coordenadas[coordenada] = []

In [10]:
len(diccionar_coordenadas)

35

In [11]:
fila  = np.array(list(data_point.iloc[0])[1:-1])
len(fila)

35

In [12]:
fila

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0.])

In [13]:
np.where(fila==1)[0]

array([16, 29], dtype=int64)

In [14]:
conexiones = np.where(fila==1)[0]
for c in conexiones:
    print(c)

16
29


In [15]:
for i in range(len(data_point)):
    #tomo las conexiones
    fila  = np.array(list(data_point.iloc[i])[1:-1])
    
    conexiones = np.where(fila==1)[0]

    for idx in conexiones:
        #almaceno las coordenadas 
        diccionar_coordenadas[listaDEpuntos[i]].append(listaDEpuntos[idx])


In [16]:
diccionar_coordenadas

{(-0.20248826666666667, -78.50040125): [(-0.20381238333333332, -78.5007766),
  (-0.2032972, -78.49823549999999)],
 (-0.21495733333333333, -78.49079993333334): [],
 (-0.20644891999999998, -78.49941296): [(-0.20728062500000002, -78.498081775),
  (-0.20768993333333333, -78.49984930000001),
  (-0.205504975, -78.499905125),
  (-0.2032972, -78.49823549999999)],
 (-0.21103442500000003,
  -78.4981667125): [(-0.21096573333333335,
   -78.49978503333334), (-0.20872455454545458, -78.49558266363637), (-0.21235037499999998,
   -78.49691080000001)],
 (-0.20301313333333335, -78.50411983333333): [(-0.20275515, -78.50347635)],
 (-0.2126993, -78.491860325): [(-0.21495733333333333, -78.49079993333334),
  (-0.21029951428571433, -78.49388817142858)],
 (-0.21029951428571433, -78.49388817142858): [(-0.2126993, -78.491860325),
  (-0.20952433333333334, -78.49254119999999),
  (-0.21137536666666668, -78.49542793333333),
  (-0.20872455454545458, -78.49558266363637)],
 (-0.20728062500000002, -78.498081775): [(-0.20

## Consulta a Google API

In [42]:
#los puntos entran como tuplas
#Latitud, longitud
#ejemplo distancdistancia_dos_puntos(pt1 = (2.5,3.5), pt2 = (8,7.3))
def distancia_dos_puntos(pt1, pt2):
    #codigo de consulta
    #Enter your source and destination city
    lat1 = str(pt1[0])
    long1 = str(pt1[1])
    lat2 = str(pt2[0])
    long2 = str(pt2[1])
    originPoint = lat1 + ',' + long1
    destinationPoint= lat2 + ',' + long2
    #Place your google map API_KEY to a variable
    apiKey = 'Your_ApiKeY'
    #Store google maps api url in a variable
    url = 'https://maps.googleapis.com/maps/api/distancematrix/json?'
    # call get method of request module and store respose object
    r = requests.get(url + 'origins=' + originPoint + '&destinations=' + destinationPoint + '&key=' + apiKey)
    #Get json format result from the above response object
    res = r.json()
    #retorna la distancia
    return res

Este `for` recorre todas las claves del diccionario y hace la consulta a las funciones.

In [44]:
tiempo_dict = {}
distancia_dict = {}

for key in diccionar_coordenadas:
    tiempo_dict[key] = []
    distancia_dict[key] = []
    for vecino in diccionar_coordenadas[key]:
        distancia_dict[key].append(distancia_dos_puntos(key,vecino))

In [45]:
distancia_dict

{(-0.20248826666666667,
  -78.50040125): [{'destination_addresses': ['Perez Guerrero Y Versalles, Quito, Ecuador'],
   'origin_addresses': ['San Gregorio &, Quito 170129, Ecuador'],
   'rows': [{'elements': [{'distance': {'text': '0.2 km', 'value': 209},
       'duration': {'text': '1 min', 'value': 62},
       'status': 'OK'}]}],
   'status': 'OK'}, {'destination_addresses': ['Av. 10 de Agosto 170526, Quito 170129, Ecuador'],
   'origin_addresses': ['San Gregorio &, Quito 170129, Ecuador'],
   'rows': [{'elements': [{'distance': {'text': '0.3 km', 'value': 257},
       'duration': {'text': '1 min', 'value': 64},
       'status': 'OK'}]}],
   'status': 'OK'}],
 (-0.21495733333333333, -78.49079993333334): [],
 (-0.20644891999999998,
  -78.49941296): [{'destination_addresses': ['Avenida Patria, Quito 170143, Ecuador'],
   'origin_addresses': ['10 de Agosto Y Patria, Quito, Ecuador'],
   'rows': [{'elements': [{'distance': {'text': '0.6 km', 'value': 615},
       'duration': {'text': '2 m

Pruebas con Api de Google

In [46]:
a = distancia_dict
with open('Consultas_googleApi.pickle', 'wb') as handle:
    pickle.dump(a, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [47]:
with open('Consultas_googleApi.pickle', 'rb') as handle:
    recuperacion = pickle.load(handle)

In [48]:
recuperacion

{(-0.20248826666666667,
  -78.50040125): [{'destination_addresses': ['Perez Guerrero Y Versalles, Quito, Ecuador'],
   'origin_addresses': ['San Gregorio &, Quito 170129, Ecuador'],
   'rows': [{'elements': [{'distance': {'text': '0.2 km', 'value': 209},
       'duration': {'text': '1 min', 'value': 62},
       'status': 'OK'}]}],
   'status': 'OK'}, {'destination_addresses': ['Av. 10 de Agosto 170526, Quito 170129, Ecuador'],
   'origin_addresses': ['San Gregorio &, Quito 170129, Ecuador'],
   'rows': [{'elements': [{'distance': {'text': '0.3 km', 'value': 257},
       'duration': {'text': '1 min', 'value': 64},
       'status': 'OK'}]}],
   'status': 'OK'}],
 (-0.21495733333333333, -78.49079993333334): [],
 (-0.20644891999999998,
  -78.49941296): [{'destination_addresses': ['Avenida Patria, Quito 170143, Ecuador'],
   'origin_addresses': ['10 de Agosto Y Patria, Quito, Ecuador'],
   'rows': [{'elements': [{'distance': {'text': '0.6 km', 'value': 615},
       'duration': {'text': '2 m